In [ ]:
import collections
import hashlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import random
from collections import Counter

In [ ]:
import pandas as pd
import os

create a dataframe object from the csv file

In [ ]:
new_dataframe = pd.read_csv(r"C:\Users\pushp\Downloads\Geolife-Trajectories-1.3\Geolife-Trajectories-1.3\Modified_Final_Dataset.csv")
new_dataframe.head()

Next few parts, we will be making our data in a usable format for feeding into our dataset

In [ ]:
def list_generate(new_dataframe,user_num):    
    user_lat = new_dataframe[new_dataframe.User_ID==user_num]['Modified_Latitude'].to_list()
    user_lon = new_dataframe[new_dataframe.User_ID==user_num]['Modified_Longitude'].to_list()
    user_timestamp = new_dataframe[new_dataframe.User_ID==user_num]['Timestamp'].to_list()
    l = len(new_dataframe[new_dataframe.user_num==57])
    user_path = [(querier_lat[i], querier_lon[i], querier_timestamp[i]) for i in range(l)]
    return user_path


For 50 infected users, we will create a list of lists where each list is the path of one infected patient

In [ ]:
querier_path = list_generate(new_dataframe,57)
infected_list = []
infected1 = list_generate(new_dataframe,94)
infected2 = list_generate(new_dataframe,150)
infected_list.append(infected1)
infected_list.append(infected2)
for i in range(50):
    infected_list.append(list_generate(new_dataframe,i))
    

In [ ]:
def position_determiner(lat, lon):
    '''Given a latitude, longitude co-ordinate, we have to find the small universe it belongs to.'''
    min_lat = 23.4377
    max_lat = 40.9999
    min_lon = 113.7947
    max_lon = 117.1832
    k = math.floor((lat-min_lat)/0.0030)
    j = math.floor((lat-min_lon)/0.0030)
    return (k,j)
    

In [ ]:
def path_divider(path):
    '''given a path (a list of tuples, (lat, lon, timestamp)), divides it into zones.
    returns: a dictionary zone->[{timestamp->(lat,lon, timestamp)},[timestamp_list]]'''
    track_dict = {}
    for elt in path:
        temp_val = position_determiner(elt[0], elt[1])
        if temp_val not in track_dict:
            track_dict[temp_val]=[{},[]]
            track_dict[temp_val][0][elt[2]]= elt
            track_dict[temp_val][1].append(elt[2])
        else:
            track_dict[temp_val][0][elt[2]]= elt
            track_dict[temp_val][1].append(elt[2])
    return track_dict


In [ ]:
class Querier():
    def __init__(self,true_path):
        self.true_path = true_path
        self.filtered_dictionary = {}
        self.intersection = []

    def filter_divider(self):
        '''returns zone->timestamp_list dictionary.'''
        self.filtered_dictionary = path_divider(self.true_path)     
        filt_dict = {e:self.filtered_dictionary[e][1] for e in self.filtered_dictionary}
        return filt_dict

    def checker(self, freq_dict, zone):
        '''given the dictionary, checks the intersection with its elements, in other words, for each of its elements in this zone,
        it checks whether the frequency in the dictionary is greater than 1.'''
        timestamp_list = self.filtered_dictionary[zone][1]
        new_all_list = [zone+(timestamp,) for timestamp in timestamp_list]
        for elt in new_all_list:
            if freq_dict[elt]>=1:
                self.intersection.append(elt)
                
    def annoncer(self):
        '''returns the self.intersection list.'''
        return self.intersection


In [ ]:
class Infected():
    def __init__(self, true_path,width, depth,epsilon,seed):
        self.width = width
        self.depth = depth
        self.epsilon = epsilon
        self.seed = seed
        self.true_path = true_path
        self.filtered_dictionary = {}
        self.new_level2_filter = {}
    #Implementing the client side algorithm which works on each particular data element. First it selects an integer randomly from 0 to depth-1, 
    #we store that value as j, next, we initialize a vector of -1, of length width, and set that jth index as 1. Then we create another vector of -1s
    #and +1s, with fixed probabilities determined by the parameter epsilon. Then we do element wise multioplication and return the 
    #result vector. 

    def Client_Side(self,d):
        '''parameters are a data element, epsilon value and a hash family, but we are using mmh3 here so didn't add that as a parameter'''
        j = np.random.randint(self.depth) 
        z_vect = np.zeros((1,self.width))
        v = z_vect-1 
        hex_num = hashlib.sha256((str(self.seed[j])+str(d)).encode('utf-8')).hexdigest()
        int_hex = int(hex_num[:5],16)
        index = int_hex % self.width
        v[0,index]=1
        val = np.exp(self.epsilon/2)
        probability_of_1 = val/(val+1) 
        probability_of_neg_1 = 1/(val+1)
        b = np.random.choice([1,-1],self.width,p=[probability_of_1,probability_of_neg_1]) 
        final_vector = v*b
        return (final_vector,j) 

    def send_flipped_bit_vector(self,zone):
        result_list = []
        for elt in self.new_level2_filter[zone]:
            result_list.append(self.Client_Side(elt))
        return result_list

    def filter_divider(self):
        '''returns: zone->timestamp_list.'''
        self.filtered_dictionary = path_divider(self.true_path)
        filt_dict = {e:self.filtered_dictionary[e][1] for e in self.filtered_dictionary}
        return filt_dict
        
    def filter2(self,filtered_dict_from_server):
        '''filtered_dict_from_server, a dictionary with zone->timestamps.
        modifies: a new_level2_filter list, and new_level2_filter_list[zone] is a list of (lat, lon, timestamp).'''
        for zone in filtered_dict_from_server:
            self.new_level2_filter[zone]=[]
            for elt in filtered_dict_from_server[zone]:
                self.new_level2_filter[zone].append(self.filtered_dictionary[zone][0][elt])



In [ ]:
class Server():
    def __init__(self, width, depth, epsilon, seed):
        self.width = width
        self.depth = depth
        self.epsilon = epsilon
        self.seed = seed 
        self.zone_list=[]
        self.new_filtered_list = []

    def Compute_Sketch_Matrix(self,D):
        '''So each element of D is a tuple where the first element is v_i and second element is j_i(which is created by the above funstion after getting 
        passed by Client_side each time), we have privacy parameter epsilon and dimensions. v_i is the a vector an it has the sma eshape as the 
        vectors returned by the Client_side algorithm'''
        val = np.exp(self.epsilon/2)
        n = len(D)
        c_epsilon = (val+1)/(val-1) 
        vec_one = np.ones((1,self.width))
        manipulated_data_matrix = np.zeros((n,self.width)) #Creatimg a matrix for x_is
        for elt in enumerate(D):
            new_vect = elt[1][0].reshape((1,self.width))
            man_vect = (c_epsilon/2)*new_vect #scalar and vector multiplication 
            half_vec_one = 0.5*vec_one 
            sum_vect = man_vect + half_vec_one
            manipulated_data_matrix[[elt[0]],:]= self.depth*sum_vect  
        M = np.zeros((self.depth,self.width))
        for elt in enumerate(D):
            for l in range(self.width): 
                M[elt[1][1],l]=M[elt[1][1],l]+manipulated_data_matrix[elt[0],l]
        return M 

    def frequency_counter(self, Sketch_Matrix, total_path_length, universe_list):
        ''' we get a sketch matrix and we try to get the frequencies for all elements in the true_path.'''
        frequency_dict = {}
        for d in universe_list:
            n = total_path_length
            frac1 = self.width/(self.width-1)
            frac2 = n/self.width
            row_sum = 0
            for i in range(self.depth):
                hex_num = hashlib.sha256((str(self.seed[i])+str(d)).encode('utf-8')).hexdigest()
                int_hex = int(hex_num[:5],16)
                index = int_hex % self.width
                row_sum = row_sum + Sketch_Matrix[i,index]
            avg_row_sum = row_sum/self.depth 
            subtraction = avg_row_sum - frac2 
            assumed_freq = frac1*subtraction 
            frequency_dict[d]=assumed_freq
        return frequency_dict

    def checker(self,querier_filtered_path, infected_patients_filtered_dict_list):
        '''gets zone-> timestamps and returns list of filtered zone-> set(timestamps) dictionaries.'''
        l = len(infected_patients_filtered_dict_list)
        for i in range(l):
            temp_dict={}
            for zone in infected_patients_filtered_dict_list[i]:
                if zone in querier_filtered_path:
                    temp_dict[zone] = set(infected_patients_filtered_dict_list[zone]) & set(querier_filtered_path[zone])
            self.new_filtered_list.append(temp_dict)#For all infected patients, it maintains a dictionary, mapping zone to timestamps.
        return self.new_filtered_list
    
    def universe_generate(self, zone, timestamp_list):
        '''generate the universe list with the timestamps.'''
        lat = zone[0]
        lon = zone[1]
        lat_list = []
        lon_list = []
        universe_list = []
        for i in np.arange(lat, lat+0.0031, 0.0001):
            for j in np.arange(lon, lon+0.0031, 0.0001):
                for time in timestamp_list:
                    universe_list.append((i, j, time))
        return universe_list

    def bit_flip_caller(self, infected_patients, querier):
        '''Will call the flip bit method on all zones of each infected patient.'''
        l = len(self.new_filtered_list)
        for i in range(l):
            for zone in self.new_filtered_list[i]:
                sketch_matrix = np.zeros((self.width,self.depth))
                for i in range(800):#run number, for doing average 
                    D = infected_patients[i].send_bit_flipped_vector(zone)
                    M = self.compute_sketch_matrix(D)
                    for row in range(self.depth):
                        for col in range(self.width):
                            sketch_matrix[row, col] = (i*sketch_matrix[row, col]+M[row, col])/(i+1)
                #now we will calculate the frequency 
                universe_list = self.universe_generate(zone, self.new_filtered_list[i][zone])
                #finally count the frequency for each element in this universe list
                n = len(self.new_filtered_list[i][zone])
                freq_dict = frequency_counter(sketch_matrix,l, universe_list)
                querier.checker(freq_dict,zone)


In [ ]:
#in the new mode, we invoke this method and the server receives this flipped bit vectors, as we want to average the 
#bit flipping thing, we run this process a mentioned number of times. and make compute sketch matrix.

In [ ]:
def driver_code(width, depth, seed, epsilon, run_number):
    '''1. Create instances of querier, server and infected patients.
       2. Call the filter method on all the participants in the server, so all the querier and the infectd patients filter out their paths 
          in multiple universes. This function will output a dictionary kepys mapping to 2 lists, one list is the co-ordinates in that universe and another list is the timestamp.
       3. Now send this filtered timestamp and zone list to the server. Form can be (zone code->timestamps in that zone)
       4. Taking the querier's timestamp against the infected patient's timestamp, server will level2 filtering and send similar form of 
          dictionary to all the infected patients and the querier, so basically finding the timestamp intersection between querier and infected patient pair
          and server is keeping the lists for his calculation.
       5. The participants, upon receiving the information, filters out the information further and start processing data from their end, aka, 
          calls the server caller method.(So in our implementation, we will be processing all infected patients one by one)
       6. The server caller method, is a common method which will be used in other parts too, when this method is called from some 
          infected patient, it asks for flipped vectors a mentioned number of times from the infected patient and upon receiving it every time,
          the server does the compute sketch matrix operation and contnues averaging it and finally gets the aggregated sketch matrix, as the server knows which 
          infected patient and from which zone, it was being called, he finally calls freq generator with this information.(Zone code, sketch matrix and server has saved the timestamp info
          for generating universe list)
       7. Thus for each caller method, server keeps generating frequency histogram and keeps aggregating it and will finally send it to the querier 
       
    '''
    querier = Querier(querier_path)
    infected_patients = []
    for infected in infected_list:
        infected_patients.append(Infected(infected, width, depth, epsilon, seed))
    server_instance = Server(width, depth, epsilon, seed)
    infected_patients_filtered_dict_list=[]
    for e in infected_patients:
        infected_patients_filtered_dict_list.append(e.filter_divider())
    querier_filter = querier.filter_divider()
    new_filtered_list = server_instance.checker(queier_filter, infected_patients_filtered_dict_list)
    l = len(new_filtered_list)
    for i in range(l):
        infected_patients[i].filter2(new_filtered_list[i])
    server_instance.bit_filp_caller()
    return querier.announcer()



In [ ]:
seed = [172,  55, 177, 191, 199,  23,  82, 126,  83,  21,   6,  69, 122,
        95,  81, 194,  80,  87, 188, 159, 107,  66,   3,  38,  52,  32,
       140,  44, 185,  20, 154,  36, 186,  99, 149, 100,  53,  58,  96,
       160, 108,  46, 150, 179,   2, 142, 151,  76,  65,  72,  59, 129,
        51,  11, 193,  79, 105,  42,  77, 104,  25, 189,  35, 125,  61,
       164, 112, 190, 106,  29,  27,  33,   9, 184,  91,  26,  93, 130,
        39, 187, 174,  88, 178, 168,  63, 198, 169, 157,  10, 111, 196,
        73,  12, 115,  78, 162,  64, 147,  54, 103]
width = 150
depth = 100
epsilon = 4.5
driver_code(width, depth, epsilon, seed)